In [100]:
%env SPARK_VERSION=3.2.1

env: SPARK_VERSION=3.2.1


In [101]:
import pydeequ
from pyspark.sql import SparkSession, Row

spark = (
    SparkSession.builder \
    .appName("TRN Database Test") \
    .config("spark.driver.extraClassPath", "/home/jovyan/work/sqljdbc42.jar") \
    .config("spark.executor.extraClassPath", "/home/jovyan/work/sqljdbc42.jar") \
    .config("spark.jars.packages", "com.amazon.deequ:deequ:2.0.1-spark-3.2") \
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord) \
    .getOrCreate()
)

print("Spark version:", spark.version)
print("PyDeequ version", pydeequ.__version__)

Spark version: 3.2.1
PyDeequ version 1.0.1


In [102]:
server = "192.168.0.26"
port = "1433"
database = "TRN"
username = "Tester"
password = "tester_tester"

jdbc_url = f"jdbc:sqlserver://{server}:{port};database={database};user={username};password={password}"

In [103]:
spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "hr.Jobs") \
    .load() \
    .createOrReplaceTempView("Jobs")

spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "hr.Employees") \
    .load() \
    .createOrReplaceTempView("Employees")

spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "hr.Dependents") \
    .load() \
    .createOrReplaceTempView("Dependents")

In [104]:
### Data Analyzers section
from pydeequ.analyzers import *

dfEmplJobs = spark.sql("""select em.*, jb.job_title, jb.min_salary, jb.max_salary 
                          from Employees em left join Jobs jb on em.job_id = jb.job_id""")

result = (
    AnalysisRunner(spark)
    .onData(dfEmplJobs)
    .addAnalyzer(Size())
    .addAnalyzer(Completeness("job_id"))
    .addAnalyzer(ApproxCountDistinct("job_title"))
    .addAnalyzer(Mean("min_salary"))
    .addAnalyzer(Mean("max_salary"))
    .addAnalyzer(Compliance("top salary", "salary >= 10000"))
    .addAnalyzer(Correlation("min_salary", "salary"))
    .run()
)

result_df = AnalyzerContext.successMetricsAsDataFrame(spark, result)
result_df.show()

+-----------+-----------------+-------------------+-------------------+
|     entity|         instance|               name|              value|
+-----------+-----------------+-------------------+-------------------+
|Mutlicolumn|min_salary,salary|        Correlation|0.26285701967746006|
|     Column|           job_id|       Completeness|                1.0|
|     Column|       top salary|         Compliance|                0.2|
|     Column|        job_title|ApproxCountDistinct|               18.0|
|     Column|       min_salary|               Mean|             5740.0|
|    Dataset|                *|               Size|               40.0|
|     Column|       max_salary|               Mean|            11650.0|
+-----------+-----------------+-------------------+-------------------+



In [105]:
### Data profiling section
from pydeequ.profiles import ColumnProfilerRunner

df = spark.sql("select * from Dependents")

result = (
    ColumnProfilerRunner(spark)
    .onData(df) 
    .run()
)

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: first_name: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 29,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 30
    },
    "histogram": [
        [
            "Helen",
            1,
            0.03333333333333333
        ],
        [
            "Kirsten",
            1,
            0.03333333333333333
        ],
        [
            "Joe",
            1,
            0.03333333333333333
        ],
        [
            "Woody",
            1,
            0.03333333333333333
        ],
        [
            "Zero",
            1,
            0.03333333333333333
        ],
        [
            "Uma",
            1,
            0.03333333333333333
        ],
        [
            "Fred",
            1,
            0.03333333333333333
        ],
        [
            "Elvis",
            1,
         

In [106]:
### Constraint Suggestions section
from pydeequ.suggestions import *

df = spark.sql("select * from jobs")

result = (
    ConstraintSuggestionRunner(spark)
    .onData(df)
    .addConstraintRule(CompleteIfCompleteRule())
    .addConstraintRule(NonNegativeNumbersRule())
    .run()
)

print(json.dumps(result, indent=2))

{
  "constraint_suggestions": [
    {
      "constraint_name": "CompletenessConstraint(Completeness(job_id,None))",
      "column_name": "job_id",
      "current_value": "Completeness: 1.0",
      "description": "'job_id' is not null",
      "suggesting_rule": "CompleteIfCompleteRule()",
      "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
      "code_for_constraint": ".isComplete(\"job_id\")"
    },
    {
      "constraint_name": "ComplianceConstraint(Compliance('job_id' has no negative values,job_id >= 0,None))",
      "column_name": "job_id",
      "current_value": "Minimum: 1.0",
      "description": "'job_id' has no negative values",
      "suggesting_rule": "NonNegativeNumbersRule()",
      "rule_description": "If we see only non-negative numbers in a column, we suggest a corresponding constraint",
      "code_for_constraint": ".isNonNegative(\"job_id\")"
    },
    {
      "constraint_name": "CompletenessConstraint(Completeness(job

In [115]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report

from pydeequ.checks import *
from pydeequ.verification import *
from pyspark.sql.functions import *


df = spark.sql("select * from jobs")

check = Check(spark, CheckLevel.Warning, "Review Check")

result = (
    VerificationSuite(spark) 
    .onData(df) 
    .addCheck(
        check.hasSize(lambda x: x > 50) 
        .hasMin("min_salary", lambda x: x == 2000.0) 
        .isComplete("job_title")  
        .isUnique("job_id") 
        .isNonNegative("min_salary") 
        .isNonNegative("max_salary"))
    .run()
)

result_df = VerificationResult.checkResultsAsDataFrame(spark, result)

# Beautify the report
print("Constraint Verification Report")
print("-------------------")
total = result_df.count()
success = result_df.filter(col("constraint_status") == "Success").count()
fail = result_df.filter(col("constraint_status") != "Success").count()
print("Total Checks: {}".format(total))
print("Passed Checks: {}".format(success))
print("Failed Checks: {}".format(fail))
print("Data Quality Score: {:.2%}".format((total - fail) / total))

print("\nFailed Checks:")
res = result_df.filter(col("constraint_status") == "Failure").collect()
if res:
    for row in res:
        print("- Constraint: {}; Status: {}; Message: {}".format(row["constraint"], row["constraint_status"], row["constraint_message"]))
else:
    print("None")

print("\nPassed Checks:")
res = result_df.filter(col("constraint_status") == "Success").collect()
if res:
    for row in res:
        print("- Constraint: {}; Status: {}; Message: {}".format(row["constraint"], row["constraint_status"], row["constraint_message"]))
else:
    print("None") 


Constraint Verification Report
-------------------
Total Checks: 6
Passed Checks: 5
Failed Checks: 1
Data Quality Score: 83.33%

Failed Checks:
- Constraint: SizeConstraint(Size(None)); Status: Failure; Message: Value: 19 does not meet the constraint requirement!

Passed Checks:
- Constraint: MinimumConstraint(Minimum(min_salary,None)); Status: Success; Message: 
- Constraint: CompletenessConstraint(Completeness(job_title,None)); Status: Success; Message: 
- Constraint: UniquenessConstraint(Uniqueness(List(job_id),None)); Status: Success; Message: 
- Constraint: ComplianceConstraint(Compliance(min_salary is non-negative,COALESCE(CAST(min_salary AS DECIMAL(20,10)), 0.0) >= 0,None)); Status: Success; Message: 
- Constraint: ComplianceConstraint(Compliance(max_salary is non-negative,COALESCE(CAST(max_salary AS DECIMAL(20,10)), 0.0) >= 0,None)); Status: Success; Message: 
